# 02 — Model Training / Selection with Baselines

In [ ]:
import pandas as pd
from src.data import load_merge
from src.features import build_features
from src.baselines import evaluate_naives, make_holdout_masks, wmae
from src.models_rf import train_rf

DF = load_merge('data')
base = evaluate_naives(DF, holdout_weeks=8)
base

In [ ]:
mod, X, y, feats = build_features(DF)
cutoff, tr_mask, te_mask = make_holdout_masks(mod, holdout_weeks=8, date_col='Date')
X_tr, y_tr = X[tr_mask], y[tr_mask]
X_te, y_te = X[te_mask], y[te_mask]
X_tr.shape, X_te.shape

In [ ]:
from sklearn.metrics import mean_absolute_error
from src.models_rf import train_rf
rf, cv_mae = train_rf(X_tr, y_tr, n_splits=5)
yhat = rf.predict(X_te)
# weights for WMAE
w = (mod[tr_mask].groupby(['Store','Dept'])['Weekly_Sales'].mean().rename('w').reset_index())
w_te = mod.loc[te_mask, ['Store','Dept']].merge(w, on=['Store','Dept'], how='left')['w'].fillna(y_tr.mean())
mae = mean_absolute_error(y_te, yhat)
from src.baselines import wmae
wmae_te = wmae(y_te.values, yhat, w_te.values)
{'cv_mae': cv_mae, 'holdout_mae': mae, 'holdout_wmae': wmae_te}